In [1]:
!pip install assemblyai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.9 MB/s eta 0:00:00


In [2]:
!pip install openai==1.2.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.9/219.9 kB 4.6 MB/s eta 0:00:00


In [3]:
import openai
from openai import OpenAI
client = OpenAI(
  api_key="bks-3f6fd9130cb4ba794b33bb22b1829f2fad3e0fbfc576cf78",
  base_url="https://api.trybricks.ai/api/providers/openai/v1",
)
def get_completion(prompt, model = "gpt-4-turbo-preview", temperature =0):
  messages = [{"role":"user", "content": prompt}]
  response = client.chat.completions.create(
      model = model,
      messages = messages,
      temperature =temperature # degree of expiration (randomness) (0 same result - 1 creative)
  )
  return response.choices[0].message.content

In [4]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [5]:
import os
import assemblyai as aai
import pandas as pd
import json


In [48]:
aai.settings.api_key = "c390f6c66e914d60be43328342231210"
names = os.listdir('/content/drive/MyDrive/2_22')
config = aai.TranscriptionConfig(
  speaker_labels=True,
  speakers_expected=2
)


In [ ]:
ns = names[210:240]
ns

## new


In [54]:
def get_words(transcript):
  speaker = []
  words = []
  start_time = []
  end_time = []
  for utterance in transcript.utterances:
    for word in utterance.words:
      speaker.append(word.speaker)
      words.append(word.text)
      start_time.append(word.start)
      end_time.append(word.end)
  result = pd.DataFrame({'Speaker': speaker, 'Utterance start time (milliseconds)': start_time, 'Utterance end time (milliseconds)': end_time,  'Utterance': words})
  return result

In [55]:
def combine_sentence(result):
  speaker = []
  text = []
  start_time = []
  end_time = []
  temp = ''
  s_time = result['Utterance start time (milliseconds)'][0]
  for i in range(len(result)):
    seg = result['Utterance'][i]
    temp = temp + " " + seg
    if (seg[-1] in ['?','.']):
      speaker.append(result['Speaker'][i])
      text.append(temp)
      start_time.append(s_time)
      end_time.append(result['Utterance end time (milliseconds)'][i])
      if (i+1 < len(result)):
        s_time = result['Utterance start time (milliseconds)'][i+1]
      temp  = ''
  result_1 = pd.DataFrame({'Speaker': speaker, 'Utterance start time (milliseconds)': start_time, 'Utterance end time (milliseconds)': end_time,  'Utterance': text})
  return result_1

In [ ]:
for n in ns:
  path = '/content/drive/MyDrive/2_22/' + n
  transcript = aai.Transcriber().transcribe(path, config)
  if transcript.utterances is None:
    result = pd.DataFrame({'Speaker': [], 'Utterance start time (milliseconds)': [], 'Utterance end time (milliseconds)': [],  'Utterance': []})
  else:
    result = get_words(transcript)
  if len(result) != 0:
    result_1 = combine_sentence(result)

    temp = [result['Speaker'][i]+ " : " + result['Utterance'][i]  for i in range(len(result))]
    d = ' '.join(temp) [:900]
    prompt = f"""I will provide you a transcript of a tutor and a student.
    There will to two speakers A and B. You task is to identify which is student and which is tutor.
    In this session, a tutor may teach students knowledge about the topic, help student solve their problem, or ask questions to check the students' understanding.
    Format your output in a JSON format with A,B as key; student, tutor as value
    """ + d
    r = get_completion(prompt)
    a = json.loads(r[8:-4])
    result_1 = result_1.replace({'Speaker': a})
    result_1.to_csv('/content/drive/MyDrive/transcript/'+n[:-4]+'.csv', index=False)
  else:
    result.to_csv('/content/drive/MyDrive/transcript/'+n[:-4]+'.csv', index=False)
  print(n)

a32c26fd-c118-48ef-a6ec-1c819e609f44.mp3
a5c846c8-2447-447d-8ef8-46b5e6db22cb.mp3
a6f9b15b-7039-436c-9b3b-6fb820eb85fa.mp3
a81f43ec-22a2-4e1d-babf-db7ae1a04093.mp3
a9cd9d98-2e1b-4565-b8db-fb88738ac0cd.mp3
2a9ce4c1-1b0d-48a8-b177-9c4970a31c0d.mp3
16413f32-d8d7-43fa-b223-a629518c2a4c.mp3
aafc7f6e-4698-410d-8960-607e5bdfefa1.mp3
55fcc4f7-24b0-4d37-8928-a4fec674f72e.mp3
af8b82ea-5fdf-4f4e-bf47-0cec704133f3.mp3


## prev

In [ ]:
for n in names:
  path = '/content/drive/MyDrive/Colab Notebooks/sampleaudio/audio/' + n
  transcript = aai.Transcriber().transcribe(path, config)
  speaker = []
  text = []
  start_time = []
  end_time = []
  for utterance in transcript.utterances:
    speaker.append(utterance.speaker)
    text.append(utterance.text)
    start_time.append(utterance.start)
    end_time.append(utterance.end)
  result = pd.DataFrame({'Speaker': speaker, 'Utterance start time (milliseconds)': start_time, 'Utterance end time (milliseconds)': end_time,  'Utterance': text})

  temp = [result['Speaker'][i]+ " : " + result['Utterance'][i]  for i in range(len(result))]
  d = ' '.join(temp) [:4097]
  prompt = f"""I will provide you a transcript of a tutor and a student.
  There will to two speakers A and B. You task is to identify which is student and which is tutor.
  Format your output in a JSON format with A,B as key; student, tutor as value
  """ + d
  r = get_completion(prompt)
  a = json.loads(r)
  result = result.replace({'Speaker': a})


  result.to_csv('/content/drive/MyDrive/Colab Notebooks/sampleaudio/transcript/'+n[:-4]+'.csv', index=False)

## task

In [ ]:
import os
names = os.listdir('/content/drive/MyDrive/Colab Notebooks/sampleaudio/transcript')

In [ ]:
names[2:]

['02c89eec-1b4f-4cb6-ae0f-ed00ae31d73f.csv',
 '9b2cf845-211b-489c-b799-fac19b192aaa.csv',
 '38a5f8bc-527e-4269-b7d3-4a16d0de4e55.csv',
 '43e4baec-0c91-47cd-be91-50ad67d268ab.csv',
 '575ada37-caf9-4dc6-9956-6d2e115bcf9f.csv',
 '9225e68c-9d52-4148-952d-70f417d77c19.csv',
 '63734916-95cc-43ee-ac5c-df0268e45b4d.csv',
 'aa642ee0-766f-42a4-bc42-9ca928ee3273.csv',
 'a7f7932a-3fa4-4797-be9c-320f61225d97.csv',
 '0e204b03-0e0c-49ef-aa70-52f5e4d08ff1.csv']

In [ ]:
t = pd.DataFrame({'file': names[2:], 'assign': ["Kevin", "Kevin", "Mufei" , "Mufei" , "Sandy",  "Sandy", "Wayne" , "Wayne" , "Yujie", "Yujie"]})
t.to_csv('/content/drive/MyDrive/Colab Notebooks/sampleaudio/transcript/tasks.csv', index=False)